### GLOH (Gradient Location and Orientation Histogram)

GLOH (Gradient Location and Orientation Histogram) is a feature extraction technique commonly used in computer vision, particularly for tasks like image classification, object recognition, and feature matching. It combines gradient-based descriptors with the spatial arrangement of the gradients to represent the local structure of an image. GLOH builds upon SIFT (Scale-Invariant Feature Transform) features, providing a more robust and distinctive descriptor for matching keypoints in images.

#### Key Points:

- GLOH is designed to capture the distribution of gradients around keypoints.

- It divides the image patch around each keypoint into spatial regions and computes gradient orientation histograms.

- GLOH features are highly invariant to scale, rotation, and partial affine transformations, making them robust for various object recognition tasks.

#### Methodology

The GLOH feature extraction process can be broken down into the following key steps:

1. Image Preprocessing
    - The image is first converted to grayscale to simplify gradient calculations.

    - If the region of interest (ROI) is available (like cancer regions in medical images), the relevant portion of the image is cropped or selected.

2. Gradient Calculation (Sobel Operators)
    - The Sobel operator is used to calculate the image gradients in both the X and Y directions. These gradients represent the intensity changes in the image.

    - The gradient magnitude indicates the strength of an edge, while the orientation indicates the edge direction.

3. Keypoint Detection (SIFT)
    - SIFT detects distinctive keypoints in the image that are invariant to scale and rotation.

    - Each keypoint is described by its position, scale, and orientation.

4. GLOH Descriptor Calculation
    - For each keypoint, a local region around it is extracted, typically using its scale and orientation.

    - The orientation of gradients within this region is calculated and divided into bins (typically 36 bins to cover the 0-360° range).

    - Histograms of gradient orientations are computed for the region around each keypoint, capturing the gradient distribution.

    - These histograms are then concatenated to form the final GLOH feature vector.

5. Normalization
    - The GLOH feature vector is normalized to make the features invariant to changes in lighting and contrast.

#### Parameters

- ksize: Size of the Gabor kernel used for filtering the image. This parameter controls the spatial extent of the kernel.

- sigma: Standard deviation of the Gaussian function used in the Gabor filter, controlling the frequency response.

- lambd: Wavelength of the sinusoidal wave in the Gabor filter, which affects the frequency of the feature.

- gamma: Spatial aspect ratio. This affects the elongation of the Gabor filter's frequency.

- psi: Phase offset in the Gabor function, shifting the frequency response.

#### Advantages
- Robustness: GLOH features are robust to scale, rotation, and partial affine transformations. This makes them effective for tasks that involve object recognition in images with varying orientations or scales.

- Invariance: The GLOH descriptor is invariant to image transformations such as rotation and scale, allowing it to match features even in rotated or zoomed-in/zoomed-out images.

- Descriptive: GLOH captures both the orientation and spatial distribution of gradients around a keypoint, which helps in distinguishing unique objects or regions in an image.

#### For further info follow the links below:

[Exploring Gradient Location Orientation Histogram (GLOH) for Image Recognition and Object Detection](https://medium.com/@vincentchung_72457/exploring-gradient-location-orientation-histogram-gloh-for-image-recognition-and-object-detection-3e3c231a5b01)

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np

# Step 1: Load the image
image_path = 'cancer.jpg'  # Specify your image path
original_image = cv.imread(image_path)

if original_image is None:
    raise FileNotFoundError(f"Image not found at path: {image_path}")

# Step 2: Extract the region of interest (ROI) - dummy example here
bbox_region_image = original_image  # Replace with actual extraction logic

# Step 3: Convert to grayscale (required for Sobel and keypoint extraction)
gray_image = cv.cvtColor(bbox_region_image, cv.COLOR_BGR2GRAY)

# Step 4: Compute gradients using Sobel filters
# Sobel filter computes the gradient in both the X and Y direction
grad_x = cv.Sobel(gray_image, cv.CV_32F, 1, 0, ksize=3)
grad_y = cv.Sobel(gray_image, cv.CV_32F, 0, 1, ksize=3)

# Step 5: Compute gradient magnitude and orientation
# Magnitude represents edge strength, orientation gives direction
mag, angle = cv.cartToPolar(grad_x, grad_y, angleInDegrees=True)

# Step 6: Detect keypoints using SIFT (Scale-Invariant Feature Transform)
sift = cv.SIFT_create()
keypoints, descriptors = sift.detectAndCompute(bbox_region_image, None)
print(f"Number of keypoints detected: {len(keypoints)}")

# Step 7: Compute GLOH (Gradient Location and Orientation Histogram) features
gloh_features = []

# Iterate over each keypoint to extract the region and compute GLOH
for kp in keypoints:
    x, y = int(kp.pt[0]), int(kp.pt[1])  # Keypoint coordinates
    scale = int(kp.size / 2)             # Scale factor (size of region around the keypoint)

    # Ensure region stays within image boundaries
    if (y - scale < 0 or y + scale >= angle.shape[0] or
        x - scale < 0 or x + scale >= angle.shape[1]):
        continue

    # Extract region of interest for GLOH histogram computation
    orientation_patch = angle[y - scale:y + scale, x - scale:x + scale]
    histogram = cv.calcHist([orientation_patch.astype(np.float32)], [0], None, [36], [0, 360])
    gloh_features.append(histogram)

# Step 8: Concatenate GLOH features into a single feature vector and normalize
if gloh_features:
    gloh_features = cv.normalize(cv.hconcat(gloh_features), None)
    print("GLOH feature vector shape:", gloh_features.shape)
    print(gloh_features)
else:
    print("No valid GLOH features were extracted.")


Number of keypoints detected: 796
GLOH feature vector shape: (36, 796)
[[0.         0.         0.         ... 0.         0.         0.00200787]
 [0.         0.         0.         ... 0.         0.         0.00050197]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.00050197 ... 0.         0.         0.00050197]
 [0.         0.         0.         ... 0.         0.         0.00050197]
 [0.         0.         0.         ... 0.         0.00050197 0.        ]]
